In [1]:
from utils.settings import ENDPOINT_ARGS, ACCOUNT_ID
from utils.generic import display_JSON

from exabyte_api_client.endpoints.jobs import JobEndpoints
from exabyte_api_client.endpoints.materials import MaterialEndpoints
from exabyte_api_client.endpoints.workflows import WorkflowEndpoints
from exabyte_api_client.endpoints.workflows import WorkflowEndpoints
from exabyte_api_client.endpoints.raw_properties import RawPropertiesEndpoints

In [2]:
ACCOUNT_ID

'2XAeZaPHoyCokZMAB'

In [2]:
job_endpoints = JobEndpoints(*ENDPOINT_ARGS)
material_endpoints = MaterialEndpoints(*ENDPOINT_ARGS)
workflow_endpoints = WorkflowEndpoints(*ENDPOINT_ARGS)
raw_property_endpoints = RawPropertiesEndpoints(*ENDPOINT_ARGS)


## Tutorial 1: Submitting a simple job

In [23]:
default_material = material_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]
default_workflow = workflow_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]

material_id = default_material["_id"]
workflow_id = default_workflow["_id"]
owner_id = default_material["owner"]["_id"]


In [24]:
config = {
    "owner": {"_id": owner_id},
    "_material": {"_id": material_id},
    "workflow": {"_id": workflow_id},
    "name": "TEST JOB",
    "compute" : job_endpoints.get_compute(cluster = 'master-production-20160630-cluster-001.exabyte.io',ppn=1,queue='SR')

}

job = job_endpoints.create(config)
#job_endpoints.submit(job["_id"])

# This works as expected

In [19]:
default_material = material_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]
default_workflow = workflow_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]

# Explicit pw.in files

In [22]:
workflow_body = default_workflow.copy()
with open("Si/pw.in", "r") as f:
    workflow_body["subworkflows"][0]["units"][0]["input"][0]["content"] = f.read()
workflow = workflow_endpoints.create(workflow_body)

material_id = default_material["_id"]
workflow_id = workflow["_id"]
owner_id = default_material["owner"]["_id"]

config = {
    "owner": {"_id": owner_id},
    "_material": {"_id": material_id},
    "workflow": {"_id": workflow_id},
    "name": "Explicit pw.in",
    "compute" : job_endpoints.get_compute(cluster = 'master-production-20160630-cluster-001.exabyte.io',ppn=1,queue='SR')
}

job = job_endpoints.create(config)


In [21]:
print(default_workflow["subworkflows"][0]["units"][0]["input"][0]["content"])

{% if subworkflowContext.MATERIAL_INDEX %}
{%- set input = input.perMaterial[subworkflowContext.MATERIAL_INDEX] -%}
{% endif -%}
&CONTROL
    calculation = 'scf'
    title = ''
    verbosity = 'low'
    restart_mode = '{{ input.RESTART_MODE }}'
    wf_collect = .true.
    tstress = .true.
    tprnfor = .true.
    outdir = {% raw %}'{{ JOB_WORK_DIR }}/outdir'{% endraw %}
    wfcdir = {% raw %}'{{ JOB_WORK_DIR }}/outdir'{% endraw %}
    prefix = '__prefix__'
    pseudo_dir = {% raw %}'{{ JOB_WORK_DIR }}/pseudo'{% endraw %}
/
&SYSTEM
    ibrav = {{ input.IBRAV }}
    nat = {{ input.NAT }}
    ntyp = {{ input.NTYP }}
    ecutwfc = {{ cutoffs.wavefunction }}
    ecutrho = {{ cutoffs.density }}
    occupations = 'smearing'
    degauss = 0.005
/
&ELECTRONS
    diagonalization = 'david'
    diago_david_ndim = 4
    diago_full_acc = .true.
    mixing_beta = 0.3
    startingwfc = 'atomic+random'
/
&IONS
/
&CELL
/
ATOMIC_SPECIES
{{ input.ATOMIC_SPECIES }}
ATOMIC_POSITIONS crystal
{{ input.ATOMIC_

# Explicit pw.in file job without material

In [26]:
workflow_body = default_workflow.copy()
with open("Si/pw.in", "r") as f:
    workflow_body["subworkflows"][0]["units"][0]["input"][0]["content"] = f.read()
workflow = workflow_endpoints.create(workflow_body)

material_id = default_material["_id"]
workflow_id = workflow["_id"]
owner_id = default_material["owner"]["_id"]

config = {
    "owner": {"_id": owner_id},
    "workflow": {"_id": workflow_id},
    "name": "Explicit pw.in no material",
    "compute" : job_endpoints.get_compute(cluster = 'master-production-20160630-cluster-001.exabyte.io',ppn=1,queue='SR')
}

job = job_endpoints.create(config)


## extract energy

In [27]:
unit_flowchart_Id = workflow['subworkflows'][0]['units'][0]['flowchartId']
total_energy = raw_property_endpoints.get_property(job['_id'], unit_flowchart_Id, 'total_energy')

In [28]:
total_energy

{'_id': 'kmjZXtfBehFXjsL9L',
 'data': {'name': 'total_energy',
  'units': 'eV',
  'value': -258.6293950171692,
  'repetition': 0},
 'owner': {'_id': '2XAeZaPHoyCokZMAB', 'slug': 'wolearyc', 'cls': 'Account'},
 'creator': {'_id': 'LSr3sjC7BE5hF5fwD', 'slug': 'wolearyc', 'cls': 'User'},
 'source': {'type': 'exabyte',
  'info': {'jobId': 'xJ9uABZ9iL9HeBT8P',
   'unitId': '139044b1-f2c2-4cd0-aa2e-1d2804afd1b8'}},
 'group': 'qe:dft:gga:pbe',
 'slug': 'total_energy',
 'schemaVersion': '2022.8.16',
 'isDefault': False,
 'createdAt': '2023-10-11T15:11:28.817Z',
 'createdBy': '0',
 'inSet': [{'_id': 'nQtFH6EKKxYJFY98d', 'slug': 'read', 'cls': 'Team'},
  {'_id': 'BzSwk5ZcXW36nbFKN', 'slug': 'admin', 'cls': 'Team'},
  {'_id': 'srRCJEpiCshbKcATp', 'slug': 'owner', 'cls': 'Team'},
  {'_id': '7CCqhi9XQ3uLCs32F', 'slug': 'read', 'cls': 'Team'},
  {'_id': 'BZCrK7WG3EPvnTook', 'slug': 'comment', 'cls': 'Team'},
  {'_id': 'iBYCnzdaFSpyrX6JJ', 'slug': 'execute', 'cls': 'Team'},
  {'_id': 'ZTAW5Lhvu2EXG2H